In [1]:
import os
import shutil
import datetime
import pandas as pd
import numpy as np
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By


In [2]:
### Set Browser
#!apt-get update 
#!apt install chromium-chromedriver

chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [3]:
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
driver.get('https://www.linkedin.com/login')

C:\Users\evren\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: use options instead of chrome_options
  """Entry point for launching an IPython kernel.


In [4]:
# Find the username/
username='evren.uluu@gmail.com'
password='Ewren..10'

uname = driver.find_element("id", "username") 
uname.send_keys(username)

# Find the password input field and send the password to the input field.
pword = driver.find_element("id", "password") 
pword.send_keys(password)
time.sleep(3)

driver.find_element(By.XPATH,'//*[@id="organic-div"]/form/div[3]/button').click()


In [5]:
### Go to Job Search
driver.find_element(By.XPATH,'//*[@id="global-nav"]/div/nav/ul/li[3]/a').click()
time.sleep(5)


In [6]:
#search Jobs : entry level data analyst easy apply job adds in Ireland
driver.get('https://www.linkedin.com/jobs/search/?currentJobId=3135481518&f_AL=true&f_E=2&f_TPR=r604800&geoId=104738515&keywords=data%20analyst&location=Ireland')
time.sleep(10)



In [7]:
driver.quit()